<a href="https://colab.research.google.com/github/jmms1/Collab/blob/main/Vincular_Codigo_SITI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Instalar las bibliotecas necesarias
!pip install fuzzywuzzy
!pip install openpyxl

import pandas as pd
from fuzzywuzzy import process
from google.colab import files

# Subir los archivos
uploaded = files.upload()

Saving JulioActividades.xlsx to JulioActividades.xlsx
Saving Codigo SITI.xlsx to Codigo SITI.xlsx


In [8]:
codigo_siti_path = list(uploaded.keys())[1]
julio_actividades_path = list(uploaded.keys())[0]

codigo_siti_df = pd.read_excel(codigo_siti_path)
julio_actividades_df = pd.read_excel(julio_actividades_path)

In [9]:
print("Columnas de Codigo SITI:", codigo_siti_df.columns.tolist())
print("Columnas de Julio Actividades:", julio_actividades_df.columns.tolist())

Columnas de Codigo SITI: ['OBSERVACION', 'CLAVE', 'OBSERVACIÓN2']
Columnas de Julio Actividades: ['Name', 'Value.contrato', 'Value.id', 'Value.rfc', 'Value.rfc2', 'Value.rfc3', 'Value.hbid', 'Value.actividad', 'Value.ventas']


In [21]:
def encontrar_coincidencia(actividad, opciones, umbral=90):
    coincidencia, score = process.extractOne(actividad, opciones)
    if score >= umbral:
        print(f"Actividad: {actividad}, Coincidencia: {coincidencia}, Score: {score}")
        return coincidencia
    else:
        # Eliminar palabras de menos de 4 letras y la palabra "comercio", luego volver a buscar coincidencias
        actividad_modificada = ' '.join([word for word in actividad.split() if len(word) >= 4 and word.lower() != 'Comercio'])
        if actividad_modificada:
            coincidencia, score = process.extractOne(actividad_modificada, opciones)
            if score >= 80:
                print(f"Actividad: {actividad}, Coincidencia: {coincidencia}, Score: {score}")
                return coincidencia
        print(f"Actividad: {actividad}, Coincidencia: Sin coincidencia, Score: {score}")
        return "Sin coincidencia"

In [22]:
# Función para encontrar la coincidencia más cercana usando fuzzy matching
#def encontrar_coincidencia(actividad, opciones):
 #   coincidencia, _ = process.extractOne(actividad, opciones)
  #  return coincidencia

# Asegurándonos de que los nombres de las columnas son correctos
actividades_siti = codigo_siti_df['OBSERVACION'].tolist()

# Aplicar la función para encontrar coincidencias
julio_actividades_df['Catalogo'] = julio_actividades_df['Value.actividad'].apply(lambda x: encontrar_coincidencia(x, actividades_siti))

# Añadir las columnas 'OBSERVACIÓN', 'CLAVE' y las columnas de Julio Actividades al nuevo DataFrame
resultado_df = pd.merge(julio_actividades_df, codigo_siti_df, how='left', left_on='Catalogo', right_on='OBSERVACION')

# Seleccionar las columnas relevantes
resultado_df = resultado_df[['Name', 'Value.contrato', 'Value.id', 'Value.rfc', 'Value.rfc2', 'Value.rfc3', 'Value.hbid', 'Value.actividad', 'Value.ventas', 'OBSERVACION', 'CLAVE']]

# Guardar el nuevo archivo
resultado_df.to_excel('Nuevo_Archivo_Con_Catalogo.xlsx', index=False)

# Descargar el archivo resultante
files.download('Nuevo_Archivo_Con_Catalogo.xlsx')

Actividad: Comercio al por mayor de productos farmacéuticos, Coincidencia: Comercio al por mayor de productos farmacéuticos, Score: 100
Actividad: Comercio al por mayor de productos farmacéuticos, Coincidencia: Comercio al por mayor de productos farmacéuticos, Score: 100
Actividad: Servicios de limpieza de inmuebles, Coincidencia: Servicios de limpieza de inmuebles, Score: 100
Actividad: Servicios de limpieza de inmuebles, Coincidencia: Servicios de limpieza de inmuebles, Score: 100
Actividad: Comercio al por mayor de desechos de papel y de cartón, Coincidencia: Fabricación de papel, Score: 86
Actividad: Comercio al por mayor de desechos de papel y de cartón, Coincidencia: Fabricación de papel, Score: 86
Actividad: Comercio al por menor en tiendas importadoras de productos de origen extranjero, Coincidencia: Instalación de productos de carpintería, Score: 86
Actividad: Comercio al por menor en tiendas importadoras de productos de origen extranjero, Coincidencia: Instalación de producto

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Sección nueva